# **Synthetic Data Generator**

Reproducible code to generate synthetic individual-level population data in the United States.

## Process census tables

The synthetic data is generated based soly on publicly available census tables from the 2010 United States Census Summary File 1. We here read and preprocess these tables.

In [2]:
import pandas as pd

filename_nhgis = "data/nhgis0003_ds172_2010_blck_grp.csv"

## P1: TOTAL POPULATION
data_nhgis = pd.read_csv(filename_nhgis, encoding="ISO-8859-1")
data_nhgis["STATEA"] = data_nhgis["GISJOIN"].str.slice(1, 3)
data_nhgis["COUNTYA"] = data_nhgis["GISJOIN"].str.slice(4, 7)
data_nhgis["TRACTA"] = data_nhgis["GISJOIN"].str.slice(8, 14)
data_nhgis["BLKGRPA"] = data_nhgis["GISJOIN"].str.slice(14, 15)
data_nhgis["GEOID10"] = data_nhgis[["STATEA", "COUNTYA", "TRACTA", "BLKGRPA"]].apply(lambda x: "".join(x), axis=1)

data_geog = data_nhgis[["STUSAB", "REGIONA", "DIVISIONA", "STATE", "STATEA", "COUNTYA", "TRACTA", "BLKGRPA", "GEOID10"]]

## P1: TOTAL POPULATION
data_p1 = data_nhgis[["H7V001"]]

## P5: HISPANIC OR LATINO ORIGIN BY RACE
data_p5 = data_nhgis[["H7Z003", "H7Z004", "H7Z005", "H7Z006", "H7Z007", "H7Z008", "H7Z009", "H7Z011", "H7Z012", "H7Z013", "H7Z014", "H7Z015", "H7Z016", "H7Z017"]]

## P12A: SEX BY AGE (WHITE ALONE)
data_p12A = pd.DataFrame()
cols = ["H9A0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9A0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12A = data_nhgis[cols]

## P12B: SEX BY AGE (Black or African American Alone)
data_p12B = pd.DataFrame()
cols = ["H9B0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9B0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12B = data_nhgis[cols]

## P12C: Sex by Age (American Indian and Alaska Native Alone)
data_p12C = pd.DataFrame()
cols = ["H9C0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9C0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12C = data_nhgis[cols]

## P12D: Sex by Age (Asian Alone)
data_p12D = pd.DataFrame()
cols = ["H9D0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9D0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12D = data_nhgis[cols]

## P12E: Sex by Age (Native Hawaiian and Other Pacific Islander Alone)
data_p12E = pd.DataFrame()
cols = ["H9E0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9E0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12E = data_nhgis[cols]

## P12F: Sex by Age Sex by Age (Some Other Race Alone)
data_p12F = pd.DataFrame()
cols = ["H9F0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9F0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12F = data_nhgis[cols]

## P12G: Sex by Age (Two or More Races)
data_p12G = pd.DataFrame()
cols = ["H9G0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9G0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12G = data_nhgis[cols]

## P12H: Sex by Age (Hispanic or Latino)
data_p12H = pd.DataFrame()
cols = ["H9H0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9H0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12H = data_nhgis[cols]

## P12I: Sex by Age (White Alone, Not Hispanic or Latino)
data_p12I = pd.DataFrame()
cols = ["H9I0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(3, 26)] + ["H9I0%s" % '{number:0{width}d}'.format(width=2, number=i) for i in range(27, 50)]
data_p12I = data_nhgis[cols]

## P43: GROUP QUARTERS POPULATION BY SEX BY AGE BY GROUP QUARTERS TYPE
data_p43 = data_nhgis[["H81004", "H81009", "H81014", "H81019", "H81024", "H81029", "H81035", "H81040", "H81045", "H81050", "H81055", "H81060"]]

cons = pd.concat([data_geog, data_p1, data_p5, data_p43, data_p12A, data_p12B, data_p12C, data_p12D, data_p12E, data_p12F, data_p12G, data_p12H, data_p12I], axis=1, join='inner')
cons.to_csv("data/usa_blck_grp_cons.csv", index=False)

## Optimization modeling

We use an optimization model to construct the synthetic data by minimizing the difference between summarized information of the synthetic population and statistics in publicly available census tables.

In [ ]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, QuadExpr, quicksum
import torch
import csv

# define all the input data for the model
filename_cons = 'data/usa_blck_grp_cons.csv'
cons = pd.read_csv(filename_cons)

# set up parameters
n1, n2, n3, n4, n5 = 3, 23, 2, 7, 2
N = n1 * n2 * n3 * n4 * n5         # number of attribute combinations: HHGQ (3) ∗ AGE (23) ∗ HISPANIC (2) ∗ RACE (7) ∗ SEX (2)

with open('data/usa_blck_grp_obj.csv', 'w', newline='') as f1:
    wr1 = csv.writer(f1)
    wr1.writerow(["GEOID10", "obj"])

    with open('data/usa_blck_grp_hist.csv', 'w', newline='') as f2:
        wr2 = csv.writer(f2)
        # set up column names
        col_names = ["GEOID10"]
        hist_names = np.empty((n1, n2, n3, n4, n5), dtype="U10")
        for k1 in range(n1):
            for k2 in range(n2):
                for k3 in range(n3):
                    for k4 in range(n4):
                        for k5 in range(n5):
                            hist_names[k1, k2, k3, k4, k5] = str(k1).zfill(2) + str(k2).zfill(2) + str(k3).zfill(2) + str(k4).zfill(2) + str(k5).zfill(2)
        col_names.extend(hist_names.flatten())
        wr2.writerow(col_names)

        for idx, row in cons.iterrows():
            GEOID10 = row["GEOID10"]

            A = torch.tensor(range(N))
            A = A.reshape([n1, n2, n3, n4, n5])

            # initialize model
            m = Model('td')
            m.Params.LogToConsole = 0

            # add objective function
            obj = QuadExpr()

            # add variables and constraints
            h = {}      ## detailed histogram (decision vairable)
            for i in range(N):
                h[i] = m.addVar(obj=0, vtype=GRB.INTEGER, lb=0, ub=row["H7V001"], name="h_%d"%(i))
            m.update()

            ## P1: TOTAL POPULATION
            q0 = cons.loc[:, cons.columns.str.startswith("H7V")].to_numpy()
            res0, col_idx = {}, 0
            hist_idx = torch.flatten(A[:, :, :, :, :]).tolist()
            res0[col_idx] = m.addVar(obj=0, vtype=GRB.INTEGER, name="res0_%d"%(col_idx))
            obj += res0[col_idx] * res0[col_idx]
            m.addConstr(res0[col_idx] == q0[idx, col_idx] - quicksum(h[i] for i in hist_idx))
            m.update()

            ## P5: HISPANIC OR LATINO ORIGIN BY RACE
            q1 = cons.loc[:, cons.columns.str.startswith("H7Z")].to_numpy()
            res1, col_idx = {}, 0
            for x in range(n3):  # hispanic
                for y in range(n4):  # race 
                    hist_idx = torch.flatten(A[:, :, x, y, :]).tolist()
                    res1[col_idx] = m.addVar(obj=0, vtype=GRB.INTEGER, name="res1_%d"%(col_idx))
                    obj += res1[col_idx] * res1[col_idx]
                    m.addConstr(res1[col_idx] == q1[idx, col_idx] - quicksum(h[i] for i in hist_idx))
                    m.update()
                    col_idx += 1

            ## P43: GROUP QUARTERS POPULATION BY SEX BY AGE BY GROUP QUARTERS TYPE
            q2 = cons.loc[:, cons.columns.str.startswith("H8")].to_numpy()
            res2, col_idx = {}, 0     
            for x in range(n5):  # sex
                for y in [0, 4, 17]:  # age
                    for z in range(1, n1):  # hhgq
                        hist_idx_two_or_more = []
                        if y == 0:
                            for i in range(4):
                                hist_idx = torch.flatten(A[z, i, :, :, x]).tolist()
                                hist_idx_two_or_more.extend(hist_idx)
                        elif y == 4:
                            for i in range(4, 17):
                                hist_idx = torch.flatten(A[z, i, :, :, x]).tolist()
                                hist_idx_two_or_more.extend(hist_idx)
                        else:
                            for i in range(17, 23):
                                hist_idx = torch.flatten(A[z, i, :, :, x]).tolist()
                                hist_idx_two_or_more.extend(hist_idx)        
                        res2[col_idx] = m.addVar(obj=0, vtype=GRB.INTEGER, name="res2_%d"%(col_idx))
                        obj += res2[col_idx] * res2[col_idx]
                        m.addConstr(res2[col_idx] == q2[idx, col_idx] - quicksum(h[i] for i in hist_idx_two_or_more))
                        m.update()
                        col_idx += 1 

            ## P12A-G: SEX BY AGE BY RACE
            q3 = cons.loc[:, cons.columns.str.startswith('H9A') | cons.columns.str.startswith('H9B') | cons.columns.str.startswith('H9C') | cons.columns.str.startswith('H9D') | cons.columns.str.startswith('H9E') | cons.columns.str.startswith('H9F') | cons.columns.str.startswith('H9G')].to_numpy()
            res3, col_idx = {}, 0   
            for x in range(n4):  # race
                for y in range(n5):  # sex
                    for z in range(n2):  # age
                        hist_idx = torch.flatten(A[:, z, :, x, y]).tolist()
                        res3[col_idx] = m.addVar(obj=0, vtype=GRB.INTEGER, name="res3_%d"%(col_idx))
                        obj += res3[col_idx] * res3[col_idx]
                        m.addConstr(res3[col_idx] == q3[idx, col_idx] - quicksum(h[i] for i in hist_idx))
                        m.update()
                        col_idx += 1

            ## P12H: SEX BY AGE BY RACE (Hispanic or Latino)
            q4 = cons.loc[:, cons.columns.str.startswith("H9H")].to_numpy()
            res4, col_idx = {}, 0   
            for x in range(n5):  # sex
                for y in range(n2):  # age
                    hist_idx = torch.flatten(A[:, y, 1, :, x]).tolist()
                    res4[col_idx] = m.addVar(obj=0, vtype=GRB.INTEGER, name="res4_%d"%(col_idx))
                    obj += res4[col_idx] * res4[col_idx]
                    m.addConstr(res4[col_idx] == q4[idx, col_idx] - quicksum(h[i] for i in hist_idx))
                    m.update()
                    col_idx += 1           

            ## P12I: SEX BY AGE BY RACE (White Alone, Not Hispanic or Latino)
            q5 = cons.loc[:, cons.columns.str.startswith("H9I")].to_numpy()
            res5, col_idx = {}, 0   
            for x in range(n5):  # sex
                for y in range(n2):  # age
                    hist_idx = torch.flatten(A[:, y, 0, 0, x]).tolist()
                    res5[col_idx] = m.addVar(obj=0, vtype=GRB.INTEGER, name="res5_%d"%(col_idx))
                    obj += res5[col_idx] * res5[col_idx]
                    m.addConstr(res5[col_idx] == q5[idx, col_idx] - quicksum(h[i] for i in hist_idx))
                    m.update()
                    col_idx += 1 

            m.setObjective(obj, GRB.MINIMIZE)
            m.optimize()

            # write histogram values
            hist_values = [GEOID10]
            var_values = [int(var.X) for var in m.getVars() if 'h' == str(var.VarName[0])]
            hist_values.extend(var_values)
            wr2.writerow(hist_values)

            # write objective values
            obj = m.getObjective().getValue()
            wr1.writerow([GEOID10, obj])
            print(idx, obj)

## Convert a matrix into a list of individuals

We list all individuals in the individual-level data matrix as a collection of individuals.

In [ ]:
import pandas as pd

filename_hist = 'data/usa_blck_grp_hist.csv'
hist = pd.read_csv(filename_hist)
hist["GEOID10"] = hist["GEOID10"].astype(str).str.zfill(12)

cols = hist.columns[1:]
with open('data/usa_microdata_all.csv', 'w', newline='') as fw:
    fw.write("YEAR,STATEA,COUNTYA,TRACTA,BLKGRPA,HTYPE,AGE,ETHN,RACE,SEX\n")

    for index, row in hist.iterrows():
        bg_id = row["GEOID10"]
        print(index)
        tract_id, county_id, state_id = bg_id[:11], bg_id[:5], bg_id[:2]
        for col in cols:
            cnt = int(row[col])
            if cnt != 0:
                ht, va, e, r, s = int(col[0:2]) + 1, int(col[2:4]) + 1, int(col[4:6]) + 1, int(col[6:8]) + 1, int(col[8:10]) + 1
                for i in range(cnt): 
                    fw.write('2010,' + state_id + ',' + county_id + ',' + tract_id + ',' + bg_id + ',' + str(ht) + ',' + str(va) + ',' + str(e) + ',' + str(r) + ',' + str(s) + '\n')

## Extract data for each state and territory

We extract the synthetic data for each state and territory in the United States.

In [16]:
import pandas as pd

filename_fips = 'data/state_fips.txt'
fips = pd.read_csv(filename_fips, delimiter='|')

filename_data = 'data/usa_microdata_all.csv'
data = pd.read_csv(filename_data)
states = data['STATEA'].unique()
for i in states:
    new = data[data['STATEA'] == i]
    name = fips[fips['STATE'] == i]['STUSAB'].values[0] + '_microdata.csv'
    new.to_csv('data/microdata_by_state/' + name, index=False)
    print(name)

AL_microdata.csv
AK_microdata.csv
AZ_microdata.csv
AR_microdata.csv
CA_microdata.csv
CO_microdata.csv
CT_microdata.csv
DE_microdata.csv
DC_microdata.csv
FL_microdata.csv
GA_microdata.csv
HI_microdata.csv
ID_microdata.csv
IL_microdata.csv
IN_microdata.csv
IA_microdata.csv
KS_microdata.csv
KY_microdata.csv
LA_microdata.csv
ME_microdata.csv
MD_microdata.csv
MA_microdata.csv
MI_microdata.csv
MN_microdata.csv
MS_microdata.csv
MO_microdata.csv
MT_microdata.csv
NE_microdata.csv
NV_microdata.csv
NH_microdata.csv
NJ_microdata.csv
NM_microdata.csv
NY_microdata.csv
NC_microdata.csv
ND_microdata.csv
OH_microdata.csv
OK_microdata.csv
OR_microdata.csv
PA_microdata.csv
RI_microdata.csv
SC_microdata.csv
SD_microdata.csv
TN_microdata.csv
TX_microdata.csv
UT_microdata.csv
VT_microdata.csv
VA_microdata.csv
WA_microdata.csv
WV_microdata.csv
WI_microdata.csv
WY_microdata.csv
PR_microdata.csv
